In [1]:
import old_helper as hp
import cv2 as cv
import numpy as np

def generateScales(img, sigma, k=2**.5, numofscales=5, dim = 3):
    scales=[]
    for i in range(numofscales):
        #find the related sigam
        newsigma = sigma*(k**i)
        scales.append(blur(img,dim, newsigma).tolist())
    return scales

def blur(img, dim, newsigma):
    gauss = hp.gaussian_nor(dim, newsigma)
    return hp.covolv(img, gauss, np.uint8)

def generateOctaves(img, numofoctaves = 2, k=2**.5, numofscales=5, dim = 3, sigma = 1):
    octaves=[]
    for i in range(numofoctaves):
        newsigma = (k ** (2*(i-1))) * sigma
        scale = 1./(2**i)
        newbaseimage = cv.resize(img, None, \
                            fx = scale, \
                            fy = scale, \
                            interpolation = cv.INTER_CUBIC)
        octaves.append(generateScales(newbaseimage,newsigma))
    return octaves

def DiffOctaves(octaves):
    diffOctaves=[]
    for octave in octaves:
        diffOctave=[]
        numofscales = len(octave)
        for i in range(numofscales-1):
            diffOctave.append(absolDiff(octave[i+1],octave[i]))
        diffOctaves.append(diffOctave)
    return diffOctaves


'''
Defining a function to avoid underflow
'''
def absolDiff(img2, img1):
    rows = len(img2)
    cols = len(img2[0])
    diff = np.zeros((rows, cols))    
    for i in range(rows):
        for k in range(cols):
            # making as type int because of underflow and realted problems
            diff[i,k]=abs(int(img2[i][k]) - int(img1[i][k]))
    return diff.astype(np.uint8).tolist()


In [2]:
# read img and convert to grey (resized)
I1 = hp.readImg_Grey_Resize(file='imgs/3_1.jpg', scale=.1)

In [3]:
numofOctaves = 2
numofScales = 5
sigma = 1
k = 2**.5
dim = 3 #gaussain dimention
octaves = generateOctaves(I1)
dogs = DiffOctaves(octaves)

In [5]:
# outputing the gaus and dog images for 2nd octave
for i in range(numofOctaves):
    for j in range(numofScales):
        cv.namedWindow('gaussed', cv.WINDOW_NORMAL)
        cv.imshow('gaussed',np.array(octaves[i][j]).astype(np.uint8))
        cv.waitKey(0)
        cv.destroyAllWindows()
        

for i in range(numofOctaves):
    for j in range(numofScales-1):
        cv.namedWindow('dog', cv.WINDOW_NORMAL)
        cv.imshow('dog',np.array(dogs[i][j]).astype(np.uint8))
        cv.waitKey(0)
        cv.destroyAllWindows()

0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4


----

# Key Point Detection

In [10]:
def keyPointDetect(diffOctaves):
    numofoctaves = len(diffOctaves)
    numofDogs = len(diffOctaves[0])
    #assuming we have more than 3 scales per octave
    
    #for each octave
    keyPts=[]
    for i in range(numofoctaves):
        diffOctave = diffOctaves[i]
        # for middle scales
        for j in range(1, numofDogs - 1):
            low_scale = diffOctave[j+1]
            mid_scale = diffOctave[j]
            upp_scale = diffOctave[j-1]
            keyPts.append(analyseScale(upp_scale,mid_scale,low_scale))
    return keyPts

            
# function assumes, 3 scales provided. 
# returns true/false matrix wrt keypts
def analyseScale(upp_scale,mid_scale,low_scale):
    rows = len(upp_scale)
    cols = len(upp_scale[0])
    KeyPoints = np.zeros((rows, cols))
    # for not boundry points
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            up_slice=np.array(upp_scale)[i-1:i+2,j-1:j+2]
            mid_slice=np.array(mid_scale)[i-1:i+2,j-1:j+2]
            low_slice=np.array(low_scale)[i-1:i+2,j-1:j+2]
            KeyPoints[i,j]=checkKeyPt(up_slice, mid_slice, low_slice)
    return KeyPoints


# function assumes, 3x3 slices provided. returns true/false
def checkKeyPt(up_slice, mid_slice, low_slice):
    keyPt = mid_slice[1][1]
    # as the number range from [0,255]
    mx = -1
    mi = 256
    rows = 3
    cols = 3
    #comparing with upper_slice
    for n_slice in [up_slice, low_slice]:
        for i in range(rows):
            for j in range(cols):
                pt = n_slice[i][j]
                mi = min(pt, mi)
                mx = max(pt, mx)
                # if keypt is between min max, return
                if (keyPt < mx and keyPt > mi):
                    return 0

    # in middle layer ignoring the mid pt.
    for i in [0,2]:
        for j in [0,2]:
            # if already more than min or smaller than max
            # return False.
            pt = mid_slice[i][j]
            mi = min(pt, mi)
            mx = max(pt, mx)
            if (keyPt < mx and keyPt > mi):
                return 0
    return 255

In [11]:
keyPts = keyPointDetect(dogs)

In [8]:
np.save("keypts", keyPts)

In [9]:
for i in range(numofOctaves):
    for j in range(numofScales-3):
        cv.namedWindow('keyPts', cv.WINDOW_NORMAL)
        cv.imshow('keyPts',np.array(keyPts[i][j]).astype(np.uint8))
        cv.waitKey(0)
        cv.destroyAllWindows()

In [17]:
l = keyPointDetect(d1)
l

[array([[0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.]]), array([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])]

In [18]:
l = [[[ 7, 11, 13, 60, 13,  4,  1],
      [56, 17, 14, 19, 23, 56,  6],
      [11, 49,  6,  8, 18, 19, 12],
      [ 7,  9, 13, 26,  3,  2, 33]],
     [[ 0,  9,  0, 26,  5,  5,  2],
      [22,  0,  0,  6,  9, 24,  0],
      [ 3, 19,  0,  2,  3,  6,  7],
      [ 5,  1,  6, 11,  4,  0, 15]],
     [[ 1,  6,  1, 10,  2,  4,  2],
      [ 8,  3,  2,  1,  4,  9,  2],
      [ 3,  5,  1,  1,  0,  1,  3],
      [ 4,  0,  4,  3,  3,  0,  7]]]
analyseScale(l[0],l[1],l[2])

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

----

# Orientation Assignment 

In [144]:
def oriAssign(octaves):
    oc = []
    for octave in octaves:
        sc = []
        numofScales = len(octave)
        # selecting images corresponding to each
        # keypoint matrix
        for l in range(2, numofScales-1):
            m = findM(octave[l])
            sc.append(m)
        oc.append(sc)
    return oc

def findM(scale):
    scale = np.array(scale)
    scale = hp.padd(scale,1)
    rows, cols = scale.shape
    r = np.zeros((rows, cols))
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            dx = int(scale[i, j+1]) - scale[i, j-1]
            dy = int(scale[i+1, j]) - scale[i-1, j]
            r[i,j] = (dx**2 + dy**2)**0.5
    return r[1:-1,1:-1].astype(np.uint8).tolist()

def thetaAssign(octaves):
    oc = []
    for octave in octaves:
        sc = []
        numofScales = len(octave)
        # selecting images corresponding to each
        # keypoint matrix
        for l in range(2, numofScales-1):
            theta = findTheta(octave[l])
            sc.append(theta)
        oc.append(sc)
    return oc

import math
def findTheta(scale):
    scale = np.array(scale)
    scale = hp.padd(scale,1)
    rows, cols = scale.shape
    r = np.zeros((rows, cols))
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            dx = int(scale[i, j+1]) - scale[i, j-1] # to avoid underflow
            dy = int(scale[i-1, j]) - scale[i+1, j]
            t = math.atan2(dy, dx)*180/np.pi
            if t < 0:
                r[i,j] = t + 360
            else:
                r[i,j] = t
    return r[1:-1,1:-1].tolist()

testing below

In [132]:
c[0][0]

[[28, 16, 40, 158, 47, 42, 27],
 [154, 73, 101, 121, 95, 47, 22],
 [61, 160, 77, 130, 184, 104, 30],
 [13, 30, 27, 104, 134, 94, 87]]

In [133]:
findM(c[0][0])

[[154, 73, 174, 121, 149, 51, 47],
 [80, 153, 60, 28, 155, 95, 47],
 [213, 45, 79, 108, 46, 161, 122],
 [67, 160, 106, 168, 184, 114, 98]]

In [136]:
np.array(findTheta(c[0][0]))

array([[275.93152692, 279.33499846, 324.57699006, 273.31094151,
        219.31629366, 246.94869908, 207.64597536],
       [335.67442476, 249.79358144, 322.37376636, 102.09475708,
        241.62447814, 220.34174589, 183.65222278],
       [ 41.38811499,  69.59011717, 112.06789956,   9.02761295,
        236.30993247, 196.97191691, 212.00538321],
       [ 63.8118893 ,  84.9993554 ,  46.13817701,  50.54299209,
         93.11084065, 114.31930861, 162.29957221]])

In [157]:
m = oriAssign(c)
theta = thetaAssign(c)
print (np.array(m[0][0]))
print (np.array(theta[0][0]))

[[ 84  93 106 118 118  76  39]
 [ 96  59  44  39  89 105  68]
 [105  55  51  52  22  91  99]
 [ 79  96 103 129 133 113  95]]
[[295.34617594 284.93141718 294.88741171 271.45635863 244.51587301
  235.0690627  220.91438322]
 [339.30454927 290.55604522 308.65980825 284.74356284 232.23692203
  219.59620864 202.42586192]
 [ 29.00169942  63.43494882  58.46520809  41.90594194 172.23483398
  193.88466769 191.65029944]
 [ 59.89869543  73.07248694  65.41842552  66.80140949  89.13847427
  120.87681308 152.63178957]]


### bucketing

In [160]:
Orientation(c, l, m, theta)

[[array([[None, None, None, None, None, None, None],
         [None, None, None, None, None, None, None],
         [None, None, None, None, None, None, None],
         [None, None, None, None, None, None, None]], dtype=object),
  array([[None, None, None, None, None, None, None],
         [None, None, None, None, None, None, None],
         [None, None, None, None, None, None, None],
         [None, None, None, None, None, None, None]], dtype=object)],
 [array([[None, None, None, None],
         [None, None, None, None]], dtype=object),
  array([[None, None, None, None],
         [None, None, None, None]], dtype=object)]]

In [159]:
def Orientation(octaves, keypts, m, theta, k=2**.5, sigma=1):
    r=[]
    for i in range(len(octaves)):
        m_oc = m[i]
        theta_oc = theta[i]
        keypts_oc = keypts[i]
        octave = octaves[i]
        er = []
        for j in range(2, len(octave)-1):
            newsigma = k**(2*i + j)*sigma
            m_sc = m_oc[j-2]
            theta_sc = theta_oc[j-2]
            keypts_sc = keypts_oc[j-2]
            scale=octave[j]
            er.append(Keyslicer(m_sc,theta_sc,keypts_sc,scale, newsigma))
        r.append(er)
    return r
        
def Keyslicer(m, theta, keypts, scale, newsigma):
    scale = hp.padd(np.array(scale), 1)
    rows, cols = scale.shape
    r = np.empty((rows,cols), dtype=object)
    # asssumtion ignoring the first 16
    for i in range(8, rows-7):
        for j in range(8, cols-7):
            if keypts[i][j] == 1:
                m_slice = np.array(m)[i-8:i+8]
                theta_slice = np.array(theta)[i-8:i+8]
                r[i,j]=bucketsToStrength(m_slice, theta_slice, newsigma)
    return r[1:-1,1:-1]

def bucketsToStrength(m_slice, theta_slice, newsigma):
    g = hp.gaussian_nor(16, newsigma)
    mg = np.dot(m_slice, g)
    # getting the histogram
    buckets=[]
    for i in range(36):
        buckets.append([])
    for i in range(16):
        for j in range(16):
            buckets[m_slice[i][j]//10].append([i,j])
    
    '''
    getting the max in the histogram but if
    we have multiple buckets with max values,
    i take the convention of selecting only first
    one
    '''
    max_bucket = []
    bucket_i = 0
    for i in range(36):
        if len(buckets[i]) > len(max_bucket):
            max_bucket = buckets[i]
            bucket_i = i
            
    m_key = 0
    for point in max_bucket:
        m_key += m_slice[point[0], point[1]] # to avoid overflow
    return [bucket_i*10 + 5, m_key]

---

# Descriptor